<a href="https://colab.research.google.com/github/yeonx/ArtificialIntelligence/blob/main/DNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall kaggle -y
!pip uninstall --upgrade.pip
!pip install kaggle==1.5.6
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

Found existing installation: kaggle 1.5.12
Uninstalling kaggle-1.5.12:
  Successfully uninstalled kaggle-1.5.12

Usage:   
  pip3 uninstall [options] <package> ...
  pip3 uninstall [options] -r <requirements file> ...

no such option: --upgrade.pip
     |████████████████████████████████| 58 kB 3.7 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72857 sha256=8e37ecd39078521359ce9a005e8af32efd8ca4415f3fede9a00b141b8b6a8c7e
  Stored in directory: /root/.cache/pip/wheels/aa/e7/e7/eb3c3d514c33294d77ddd5a856bdd58dc9c1fabbed59a02a2b
Successfully built kaggle


In [2]:
!kaggle competitions download -c 2021-ai-w3-p1

  0% 0.00/26.3k [00:00<?, ?B/s]
100% 26.3k/26.3k [00:00<00:00, 20.3MB/s]


In [3]:
!ls

2021-ai-w3-p1.zip  drive  sample_data


In [4]:
!unzip 2021-ai-w3-p1.zip

Archive:  2021-ai-w3-p1.zip
  inflating: submit_sample.csv       
  inflating: test.csv                
  inflating: train.csv               


In [192]:
import pandas as pd
import numpy as np

train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
submit=pd.read_csv("submit_sample.csv")

In [193]:
train

,year,avgTemp,minTemp,maxTemp,rainFall,avgPrice
0,20100101,-4.9,-11.0,0.9,0.0,2123
1,20100102,-3.1,-5.5,5.5,0.8,2123
2,20100103,-2.9,-6.9,1.4,0.0,2123
3,20100104,-1.8,-5.1,2.2,5.9,2020
4,20100105,-5.2,-8.7,-1.8,0.7,2060
...,...,...,...,...,...,...
2552,20161227,1.2,-3.2,4.5,2.1,4226
2553,20161228,-1.4,-6.5,4.8,0.0,4200
2554,20161229,-1.3,-4.5,2.1,0.2,4224
2555,20161230,-1.4,-7.0,4.6,0.0,4219


In [194]:
x=train.drop(["avgPrice"],axis=1)
y=train["avgPrice"]

In [195]:
x["year"]=x["year"]//10000 % 100
test["year"]=test["year"]//10000 % 100

In [196]:
x["year"]

0       10
1       10
2       10
3       10
4       10
        ..
2552    16
2553    16
2554    16
2555    16
2556    16
Name: year, Length: 2557, dtype: int64

In [197]:
from sklearn.preprocessing import RobustScaler

transformer = RobustScaler().fit(x)

x_std=transformer.transform(x)
test_std=transformer.transform(test)

In [494]:
import torch
import random
seed=777
torch.manual_seed(seed)

device='cuda' if torch.cuda.is_available() else 'cpu'
if device=='cuda':
  torch.cuda.manual_seed_all(seed)

In [465]:
x=np.array(x)
test=np.array(test)
y=np.array(y)
x=torch.FloatTensor(x)
test=torch.FloatTensor(test)
y=torch.FloatTensor(y)

In [466]:
x.shape

torch.Size([2557, 5])

In [495]:
linear1=torch.nn.Linear(5,10)
linear2=torch.nn.Linear(10,10)
linear3=torch.nn.Linear(10,1)
relu=torch.nn.ReLU()

In [496]:
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)

Parameter containing:
tensor([[-0.4559, -0.6466, -0.6108,  0.0081,  0.1784,  0.4520, -0.3005, -0.4824,
         -0.0845,  0.5084]], requires_grad=True)

In [497]:
model=torch.nn.Sequential(linear1,relu,linear2,relu,linear3).to(device)
optim = torch.optim.Adam(model.parameters(), lr=0.5)

In [498]:
for stop in range(40):
  optim.zero_grad()
  h=model(x)
  cost=torch.mean((h-y)**2).to(device)
  cost.backward()
  optim.step()
  if stop%10==0:
    print(stop,cost.item())

0 12056525.0
10 2685156.75
20 2219648.25
30 2022884.875


In [499]:
pred=list()

with torch.no_grad():
  model.eval()
  y_test=model(test)
  a,b=torch.max(y_test,dim=1)
  pred.extend(a.tolist())

In [500]:
submit["Expected"]=pred

In [501]:
submit

,Id,Expected
0,0,3773.809570
1,1,3614.233398
2,2,3521.585938
3,3,4008.544678
4,4,3908.633301
5,5,3857.852051
6,6,3531.829102
7,7,3920.982910
8,8,3656.575928
9,9,3974.276855


In [502]:
submit.to_csv("submit.csv",index=False)

In [503]:
!kaggle competitions submit -c 2021-ai-w3-p1 -f submit.csv -m "Message"

100% 473/473 [00:02<00:00, 160B/s]
Successfully submitted to 배추 가격 예측